In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.svm import SVR
import os
from sys import platform
from sklearn.linear_model import Ridge, Lasso, LinearRegression # these are lines
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
from sklearn.datasets import make_regression

C:\Users\yoith\AppData\Local\Temp\ipykernel_12444\1575920758.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [6]:
df_train.sample(10)

,id,carat,cut,color,clarity,depth,table,x,y,z,price
9154,9154,1.50,Premium,I,SI1,62.3,58.0,7.33,7.22,4.53,6888
4032,4032,0.92,Ideal,J,VS2,61.1,57.0,6.31,6.22,3.83,3215
34874,34874,0.90,Very Good,F,SI2,63.4,58.0,6.08,6.01,3.83,3556
18177,18177,0.70,Very Good,G,VS2,62.7,57.0,5.65,5.70,3.56,2507
20771,20771,0.30,Ideal,G,VS2,60.8,57.0,4.38,4.33,2.65,630
34531,34531,1.06,Ideal,E,VS1,62.1,57.0,6.49,6.52,4.04,8413
6148,6148,1.13,Good,G,I1,63.4,58.0,6.58,6.61,4.18,3127
26759,26759,0.31,Premium,G,VVS1,61.2,59.0,4.34,4.32,2.65,1046
21898,21898,1.27,Ideal,I,SI1,62.4,56.0,6.95,6.90,4.32,5690
37284,37284,0.52,Premium,D,VS2,60.6,56.0,5.25,5.22,3.17,1919


In [3]:
df_train.describe()

,id,carat,depth,table,x,y,z,price
count,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000
mean,20227.000000,0.800535,61.749759,57.456757,5.736805,5.739206,3.542314,3952.091212
std,11678.496907,0.475428,1.437254,2.230407,1.124044,1.123658,0.710253,4002.061273
min,0.000000,0.200000,43.000000,43.000000,0.000000,0.000000,0.000000,326.000000
25%,10113.500000,0.400000,61.000000,56.000000,4.720000,4.730000,2.910000,956.000000
50%,20227.000000,0.700000,61.800000,57.000000,5.700000,5.710000,3.530000,2416.000000
75%,30340.500000,1.040000,62.500000,59.000000,6.540000,6.540000,4.040000,5352.000000
max,40454.000000,5.010000,79.000000,79.000000,10.740000,31.800000,31.800000,18823.000000


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       40455 non-null  int64  
 1   carat    40455 non-null  float64
 2   cut      40455 non-null  object 
 3   color    40455 non-null  object 
 4   clarity  40455 non-null  object 
 5   depth    40455 non-null  float64
 6   table    40455 non-null  float64
 7   x        40455 non-null  float64
 8   y        40455 non-null  float64
 9   z        40455 non-null  float64
 10  price    40455 non-null  int64  
dtypes: float64(6), int64(2), object(3)
memory usage: 3.4+ MB


In [6]:
df_train["cut"].unique()

array(['Very Good', 'Premium', 'Ideal', 'Fair', 'Good'], dtype=object)

In [7]:
cut_mapping = {'Fair': 0, 'Good': 1, 'Very Good': 2, 'Premium': 3, 'Ideal': 4}

df_train['cut_encoded']= df_train['cut'].map(cut_mapping)

df_train.drop('cut', axis=1, inplace=True)

df_train


,id,carat,color,clarity,depth,table,x,y,z,price,cut_encoded
0,0,0.90,H,VS2,62.6,57.0,6.11,6.09,3.82,4662,2
1,1,0.33,H,VS1,61.9,58.0,4.45,4.43,2.75,743,3
2,2,1.20,H,VS1,61.1,58.0,6.84,6.88,4.19,8039,4
3,3,1.64,J,SI1,62.6,59.0,7.45,7.51,4.68,7739,2
4,4,1.51,I,SI2,60.6,60.2,7.32,7.37,4.45,7441,2
...,...,...,...,...,...,...,...,...,...,...,...
40450,40450,1.53,E,SI1,63.5,58.0,7.29,7.35,4.65,11303,1
40451,40451,1.13,E,VS1,61.1,59.0,6.72,6.69,4.10,8353,3
40452,40452,2.04,I,SI1,57.7,64.0,8.39,8.31,4.82,11820,1
40453,40453,0.41,F,VS1,62.0,56.0,4.77,4.81,2.97,1007,4


In [8]:
df_train.drop('id', axis=1, inplace=True)


In [9]:
df_train["color"].unique()

array(['H', 'J', 'I', 'E', 'F', 'D', 'G'], dtype=object)

In [10]:

color_mapping = {'D': 6, 'E': 5, 'F': 4, 'G': 3, 'H': 2, 'I': 1, 'J': 0}

df_train['color_encoded']= df_train['color'].map(color_mapping)

df_train.drop('color', axis=1, inplace=True)

df_train


,carat,clarity,depth,table,x,y,z,price,cut_encoded,color_encoded
0,0.90,VS2,62.6,57.0,6.11,6.09,3.82,4662,2,2
1,0.33,VS1,61.9,58.0,4.45,4.43,2.75,743,3,2
2,1.20,VS1,61.1,58.0,6.84,6.88,4.19,8039,4,2
3,1.64,SI1,62.6,59.0,7.45,7.51,4.68,7739,2,0
4,1.51,SI2,60.6,60.2,7.32,7.37,4.45,7441,2,1
...,...,...,...,...,...,...,...,...,...,...
40450,1.53,SI1,63.5,58.0,7.29,7.35,4.65,11303,1,5
40451,1.13,VS1,61.1,59.0,6.72,6.69,4.10,8353,3,5
40452,2.04,SI1,57.7,64.0,8.39,8.31,4.82,11820,1,1
40453,0.41,VS1,62.0,56.0,4.77,4.81,2.97,1007,4,4


In [11]:
df_train["clarity"].unique()

array(['VS2', 'VS1', 'SI1', 'SI2', 'VVS1', 'VVS2', 'IF', 'I1'],
      dtype=object)

In [12]:

clarity_mapping = {'IF': 7, 'VVS1': 6, 'VVS2': 5, 'VS1': 4, 'VS2': 3, 'SI1': 2, 'SI2': 1, 'I1': 0}

df_train['clarity_encoded'] = df_train['clarity'].map(clarity_mapping)

df_train.drop('clarity', axis=1, inplace=True)


In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   carat            40455 non-null  float64
 1   depth            40455 non-null  float64
 2   table            40455 non-null  float64
 3   x                40455 non-null  float64
 4   y                40455 non-null  float64
 5   z                40455 non-null  float64
 6   price            40455 non-null  int64  
 7   cut_encoded      40455 non-null  int64  
 8   color_encoded    40455 non-null  int64  
 9   clarity_encoded  40455 non-null  int64  
dtypes: float64(6), int64(4)
memory usage: 3.1 MB


In [14]:
def small_pipeline (df):
    
    # 0. Transforming
    
    # 1. Variables: features & target
    X = df_train.drop("price", axis = 1)
    y = df_train["price"]
    
    # 2. Tran, test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
    
    # 3. Fit the models
    models = {
        "lr": LinReg(),
        "ridge": Ridge(), #lr similar
        "lasso": Lasso(), # lr similar 
        "sgd": SGDRegressor(),
        "knn": KNeighborsRegressor(),
        "grad": GradientBoostingRegressor(),
        "svr": SVR() #potato chip
    }

    for model in models.values():
        model.fit(X_train, y_train)
    
    # 4. Predict
    new_list = []
    for name, model in models.items():
        y_pred = model.predict(X_test)
    
    
    # 5. Evaluating the models
    
        rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
        
        dict_ = {
            "rmse": rmse,
        }
        
        new_list.append(dict_)
        
        df = pd.DataFrame(new_list).sort_values(by="rmse", ascending=True)    
    return df

In [15]:
from sklearn.linear_model import LinearRegression as LinReg
from sklearn import metrics

In [16]:
lr = LinReg()

In [17]:
small_pipeline(df_train)

C:\Users\yoith\miniconda3\envs\ironhack\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.285e+09, tolerance: 5.167e+07
  model = cd_fast.enet_coordinate_descent(


,rmse
5,6.439702e+02
4,9.401463e+02
2,1.245845e+03
1,1.316091e+03
0,1.320412e+03
6,4.284657e+03
3,4.257161e+08


In [18]:
from sklearn.ensemble import GradientBoostingRegressor

X_train = df_train.drop(columns=['price'])
y_train = df_train['price']

model = GradientBoostingRegressor()
model.fit(X_train, y_train)


GradientBoostingRegressor()

In [19]:
df_test

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,1.50,Ideal,G,VS1,61.3,53.9,7.37,7.42,4.53
1,1,1.26,Premium,E,SI2,62.9,59.0,6.85,6.82,4.30
2,2,0.39,Good,H,VS2,63.1,58.0,4.63,4.66,2.93
3,3,1.20,Very Good,I,SI1,61.2,62.0,6.75,6.84,4.16
4,4,0.31,Ideal,E,VS2,61.6,57.0,4.35,4.38,2.69
...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.40,Ideal,H,VVS2,62.2,56.0,4.70,4.75,2.94
13481,13481,0.35,Ideal,E,VVS2,61.0,57.0,4.54,4.58,2.78
13482,13482,0.90,Very Good,G,VS2,60.2,59.0,6.22,6.28,3.76
13483,13483,1.08,Very Good,F,SI1,58.2,63.0,6.69,6.78,3.92


In [20]:

def preprocess_data(df):
    
    df.drop('id', axis=1, inplace=True)
    
    cut_mapping = {'Fair': 0, 'Good': 1, 'Very Good': 2, 'Premium': 3, 'Ideal': 4}
    df['cut_encoded'] = df['cut'].map(cut_mapping)
    df.drop('cut', axis=1, inplace=True)

    color_mapping = {'D': 6, 'E': 5, 'F': 4, 'G': 3, 'H': 2, 'I': 1, 'J': 0}
    df['color_encoded'] = df['color'].map(color_mapping)
    df.drop('color', axis=1, inplace=True)

    clarity_mapping = {'IF': 7, 'VVS1': 6, 'VVS2': 5, 'VS1': 4, 'VS2': 3, 'SI1': 2, 'SI2': 1, 'I1': 0}
    df['clarity_encoded'] = df['clarity'].map(clarity_mapping)
    df.drop('clarity', axis=1, inplace=True)

    return df


df_test = preprocess_data(df_test)


In [21]:

predictions = model.predict(df_test)

df_predictions = pd.DataFrame({'id': df_test.index, 'price': predictions})

df_predictions


,id,price
0,0,14095.632365
1,1,6340.385210
2,2,673.367249
3,3,5078.528293
4,4,853.490377
...,...,...
13480,13480,935.847338
13481,13481,1029.768593
13482,13482,4218.787679
13483,13483,5527.552486


In [22]:
df_predictions.to_csv("my_submission.csv", index=False)
